# Coastal Pioneer Hydrography

Daily mean of salinity and temperature on 01 May 2015 for the following profiler moorings:
- CP04OSPM
- CP02PMUO
- CP02PMUI: *NO DATA in 01 May 2015, removed from list*
- CP02PMCO
- CP02PMCI

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
import warnings
warnings.simplefilter('ignore')
os.environ['PYTHONWARNOINGS'] = 'ignore'

plt.rcParams.update({'figure.figsize':(15,5),'font.size':18})

In [2]:
dataDir = f'/scratch2/shared/ooi-pioneer'

pmDirList = []
pmDirList.append(dataDir + '/20201202T013321038Z-CP04OSPM-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered')
pmDirList.append(dataDir + '/20201202T032723712Z-CP02PMUO-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered')
#pmDirList.append(dataDir + '/20201202T032912160Z-CP02PMUI-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered')
pmDirList.append(dataDir + '/20201202T033005656Z-CP02PMCO-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered')
pmDirList.append(dataDir + '/20201202T033239959Z-CP02PMCI-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered')

pmNameList = ['CP04OSPM', 'CP02PMUO', 'CP02PMCO', 'CP02PMCI']
pmNameList = ['OSPM', 'PMUO', 'PMCO', 'PMCI']

In [3]:
%%time
dsList = []
for pmDir in pmDirList:
    dsList.append(xr.open_mfdataset(f'{pmDir}/*.nc').swap_dims({'obs':'time'}))

CPU times: user 731 ms, sys: 314 ms, total: 1.05 s
Wall time: 1.16 s


roughly 1M points at the same lat lon over time, varying in depth, for each profiler mooring.

### Slice a single day: 01 May 2015

We save a smaller time interval from the global time. We will take the time mean of that interval to create the vertical profile. 

In [4]:
ds01May15List = []
zMax = []

for ds in dsList:
    ds01May15List.append(ds.sel(time=slice("2015-05-01", "2015-05-02T00:00")))
    zMax.append(max(ds01May15List[-1].coords['depth'].values))

#### Convert measured to potential temperature

To account for the pressure effects, a variable called potential temperature, denoted θ, is traditionally used in oceanography. The potential temperature of a water parcel is the temperature that would be measured if the water parcel were enclosed in a bag (to prevent the loss or gain of any salt) and brought to the ocean surface adiabatically (i.e., without exchanging any heat with its surroundings).

p0 is at the waters surface usually 0dbar.

[Calculates potential temperature as per UNESCO 1983 report](https://pythonhosted.org/seawater/eos80.html)


Practical Salinity is PSU

In [5]:
import seawater as sw

In [6]:
for ds01May15 in ds01May15List:
    tmp = sw.eos80.ptmp(ds01May15['practical_salinity'],\
             ds01May15['ctdpf_ckl_seawater_temperature'],\
             ds01May15['ctdpf_ckl_seawater_pressure'])
    ds01May15['THETA'] = xr.DataArray( data=tmp, dims=['time'],\
                           coords=ds01May15.coords,\
                           attrs=dict(description="Potential temperature",\
                                      units="degC") )
    
    soundSpeed = sw.eos80.svel(ds01May15['practical_salinity'],\
             ds01May15['ctdpf_ckl_seawater_temperature'],\
             ds01May15['ctdpf_ckl_seawater_pressure'])
    ds01May15['soundSpeed'] = xr.DataArray( data=soundSpeed, dims=['time'],\
                                            coords=ds01May15.coords,\
                                            attrs=dict(description="Sound Velocity",\
                                                       units="m/s") )

### Apply mean over all depth intervals

The mean values are assigned to the bottom bound of the interval. 

ie. for the interval from 26 to 27 m, the time mean value is stored at 26 m depth. 

In [7]:
def meanOnDepthIntervals(scalarVal,Depth):
    DList = []
    for ds01May15, zM in zip(ds01May15List, Depth):
        binEdge = np.linspace( 0, np.ceil(zM), num=int(np.ceil(zM)+1) )
        valD = dict.fromkeys(binEdge, [])
        dz = binEdge[1]-binEdge[0]
    
        for z in valD:
            tmpVal = ds01May15[scalarVal].where((ds01May15.coords['depth'] >= z) & \
                                                (ds01May15.coords['depth'] < z+dz) )
            #if (ds01May15.attrs["subsite"] == 'CP04OSPM'):
            #    print(tmpVal.mean('time').values.tolist())
            valD[z] = tmpVal.mean('time').values.tolist()
        
        DList.append(valD)
    return DList

In [9]:
%%time
saltDList = meanOnDepthIntervals('practical_salinity',zMax)
tempDList = meanOnDepthIntervals('THETA',zMax)
soundSpDList = meanOnDepthIntervals('soundSpeed',zMax)

CPU times: user 1min 9s, sys: 1.28 s, total: 1min 10s
Wall time: 1min 3s


#### Save the T, S, Sound speed profiles 

In [ ]:
for ds in ds01May15List:
    ds.to_netcdf("../data/ncFiles/%s_OOI_SSP_01May15.nc" % ds01May15.attrs["subsite"] )

## Plot salinity and temperature profiles

In [ ]:
plt.rcParams.update({'font.size':24})
plt.figure(figsize=(11,15), dpi=360)
for saltD,tempD,soundSpD, name in zip(saltDList,tempDList,soundSpDList, pmNameList):
    plt.subplot(1,3,1)
    plt.plot(saltD.values(), saltD.keys(), label=name, linewidth=2)
    plt.ylim(421,0)
    plt.legend(fontsize=20)
    plt.xlabel('Salinity [psu]')
    plt.ylabel('Depth [m]')
    
    plt.subplot(1,3,2)
    plt.plot(tempD.values(), tempD.keys(), label=name, linewidth=2)
    plt.ylim(421,0) 
    plt.xlabel(r'Temperature [$^\circ C$]')

    plt.subplot(1,3,3)
    plt.plot(soundSpD.values(), soundSpD.keys(), label=name, linewidth=2)
    plt.ylim(421,0) 
    plt.xlabel(r'Sound Speed [$m/s$]')
    
plt.suptitle('01 May 2015')
#plt.show()
#plt.savefig("../img/pioneer_15-05-01_TSprofiles.png")

### Add ECCO T,S

In [ ]:
eccoDs = xr.open_mfdataset('../data/eccov4r4_daily_15-05-01_TS.nc')
D=660 # [m]
saltDList.append('ecco');tempDList.append('ecco');soundSpDList.append('ecco');pmNameList.append('ecco')

In [ ]:
#calc sound speed with oceanAcouPy
import oceanAcouPy as oa

In [ ]:
eccoDs['soundSpeed'] = oa.soundSpeed.cSTD(eccoDs['THETA'], eccoDs['SALT'], eccoDs.Z, eqn='leroy08')

In [ ]:
eccoDs

In [ ]:
plt.figure(figsize=(18,15), dpi=1080)
for saltD, tempD, soundSpD, name in zip(saltDList,tempDList,soundSpDList, pmNameList):
    plt.subplot(1,3,1)
    if (saltD == 'ecco'):
        plt.plot(eccoDs['SALT'].T, -1*eccoDs.Z, c='k', label='ECCOv4r4', linewidth=3)
    else:
        plt.plot(saltD.values(), saltD.keys(), label=name, linewidth=2)
    
    plt.ylim(D,0)
    plt.xlabel('Salinity [psu]')
    plt.ylabel('Depth [m]')
    
    plt.subplot(1,3,2)
    if (tempD == 'ecco'):
        plt.plot(eccoDs['THETA'].T, -1*eccoDs.Z, c='k', label='ECCOv4r4', linewidth=3)
    else:
        plt.plot(tempD.values(), tempD.keys(), label=name, linewidth=2)
    plt.ylim(D,0) 
    plt.xlabel(r'Temperature [$^\circ C$]')
    
    plt.subplot(1,3,3)
    if (soundSpD == 'ecco'):
        plt.plot(eccoDs['soundSpeed'].T, -1*eccoDs.Z, c='k', label='ECCOv4r4', linewidth=3)
    else:
        plt.plot(soundSpD.values(), soundSpD.keys(), label=name, linewidth=2)
    plt.ylim(D,0) 
    plt.xlabel(r'Sound Speed [$m/s$]')
    plt.legend(fontsize=20)
    
plt.suptitle('01 May 2015')

plt.savefig("../img/ecco+pioneer_15-05-01_TSSoundSp-profiles.png")

In [ ]:
%%time
svelDList = []
for ds01May15, zM in zip(ds01May15List, zMax):
    binEdge = np.linspace( 0, np.ceil(zM), num=int(np.ceil(zM)+1) )
    svelD = dict.fromkeys(binEdge, [])
    dz = binEdge[1]-binEdge[0]
    
    for z in svelD:
        tmpS = ds01May15['soundSpeed'].where((ds01May15.coords['depth'] >= z) & \
                                                (ds01May15.coords['depth'] < z+dz) )

        svelD[z] = tmpS.mean('time').values
        
    svelDList.append(svelD)

In [ ]:
plt.rcParams.update({'font.size':24})
plt.figure(figsize=(11,15), dpi=360)
for svelD,tempD, name in zip(svelDList,tempDList, pmNameList):
    plt.subplot(1,2,1)
    plt.plot(svelD.values(), svelD.keys(), label=name, linewidth=2)
    plt.ylim(421,0)
    plt.legend(fontsize=20)
    plt.xlabel('Sound Speed [m/s]')
    plt.ylabel('Depth [m]')
    
    plt.subplot(1,2,2)
    plt.plot(tempD.values(), tempD.keys(), label=name, linewidth=2)
    plt.ylim(421,0) 
    plt.xlabel(r'Temperature [$^\circ C$]')
    
plt.suptitle('01 May 2015')
#plt.savefig("../img/pioneer_15-05-01_cTprofiles.png")

<sub>Written by I. Escobar on 10 December 2020</sub>